<a href="https://colab.research.google.com/github/hussain0048/Machine-Learning/blob/master/Auto_Model_Training_and_Evaluation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**

Automated machine learning (AutoML) enables you to automate machine learning processes. There is a wide range of automation tools, created for different purposes. In general, machine learning tools are typically categorized into three categories of benefits—improving productivity, enforcing standardization, and promoting democratization.

This article explains what is AutoML, key advantages of AutoML, and what you can actually automate in machine learning. Including a review of seven top AutoML tools [12]

Automated machine learning (AutoML) is the use of automation, machine learning, and best practices to make ML accessible to a wider range of users. It is designed to help organizations speed up their training and use of ML models despite limited or a lack of access to ML experts, such as data scientists.

AutoML enables organizations to build and deploy models using templates, frameworks, and predefined processes. This speeds time to completion and helps ensure that models are as functional as possible[12]

**Advantages of AutoML**

There are three main advantages to incorporating AutoML. These are:

- Productivity—automation reduces the manual resources needed to monitor and perform repetitive ML tasks. This frees teams to focus on model refinement and packaging.
- Standardization—automated pipelines help reduce the chance of configuration errors and ensure that training and tests are performed uniformly.
- Democratization—AutoML lowers the barrier to entry for organizations with little to no ML expertise. This increases competitiveness and can increase innovation.

**What Can We Automate in Machine Learning?**

Although it might be nice to fully automate ML processes, that is not what AutoML does. Rather, it focuses on a few areas with high repetition. These areas include the following.

**Hyperparameter optimization**

Hyperparameters are values that define how a model is trained and directly impact the accuracy of training. For example, the rate of learning, number of hidden units or layers, activations functions, or the number of epochs.

Hyperparameter optimization is a process of running your algorithm with different combinations of hyperparameters. This is done to determine which set produces the most accurate result. This process is performed with search algorithms, such as grid search, random search, or Bayesian methods.[12]

**Model selection**

Model selection is a process of narrowing down candidate models for your training. It applies models to data to determine which model produces the optimum combination of performance, maintainability, and complexity. It is impacted by what resources are available to you and determines how your ML pipeline is structured.

Model selection is automated similarly to hyperparameter optimization, by running through multiple models and comparing the results. It often involves the same sort of search methods as hyperparameter optimization as well. However, it may also include the use of more extensive filters, including Bayesian Information Criterion (BIC) or Akaike Information Criterion (AIC).[12]

**Feature selection**

Features are the data points used by your model to classify datasets. Feature selection involves defining how many features are going to be used, which features, and how defined those features are. The features you select directly impact the complexity of your model and its performance.

Feature selection is automated through the application of algorithms during testing. For example, by using wrapper, filter, or embedded algorithms. During testing, features and combinations of features are evaluated to determine how accurate classifications or predictions are based on those features.

**Top AutoML Tools**

**Run:AI** is a proprietary platform for automating machine learning infrastructure. In terms of AutoML the platform offers controls for automating resource management, as well as workload orchestration for your entire machine learning infrastructure.

You can use Run:AI to pool GPU compute resources, set up GPU quotas, and continuously change resource allocation. These features enable you to actually optimize your compute resources, and ensure even highly intensive deep learning models consume resources at scale.[12]

**2. AutoKeras**

AutoKeras is an open source library based on Keras that you can use for classification and regression of images, text, and structured data. It enables you to use pre-built blocks to construct a model, leaving you to focus on high-level architecture. AutoKeras supports use with Python 3.5 and up and TensorFlow 2.1.0 and up.[12]

**3. Auto-WEKA**

Auto-WEKA is an open source library that you can use to optimize your hyperparameter selection. It uses Bayesian optimization to select a learning algorithm and hyperparameters from those available in the WEKA package. Auto-WEKA has the same requirements as WEKA and includes a graphical user interface (GUI) for ease of use.[12]

**4. DataRobot Automated Machine Learning**

DataRobot is a proprietary platform that you can use to automate and optimize model creation. It is designed for end-to-end support of model development, training, and deployment.

DataRobot provides a range of features, including for data formatting, feature engineering, model selection, hyperparameter tuning, and monitoring. It also offers pretrained models, a data catalog, and a user friendly GUI with visualizations of the entire training and deployment process.[12]

**5. H20 AutoML**

H2O is an open source platform for ML that is distributed and runs in-memory. It supports a wide range of ML and statistical algorithms, including deep learning, generalized linear models, and gradient boosted machines. While H2O is not automated, it includes a paid add-on, called H2O AutoML.

H20 AutoML enables you to train and tune models with automatic feature selection and extraction, hyperparameter optimization, and use of ensembles (multiple models for greater performance). You can use H20 AutoML from a web GUI. It integrates with Hadoop, Spark, and Kubernetes.

**6. MLBox**

MLBox is an open source Python library that you can use to automate many aspects of model training. These aspects include data preprocessing, feature selection, and hyperparameter optimization. It also includes predictive models for regression and classification, such as LightGBM, Stacking, and Deep Learning.

**7. auto-sklearn**

auto-sklearn is an open source toolkit based on scikit-learn that you can use to perform model selection, feature engineering, and hyperparameter tuning. It includes features that enable you to leverage Bayesian optimization, ensembles, and meta-learning for more accurate models and training.

When using auto-sklearn, you can restrict the time and memory limits of scikit-learn, restrict your searchspace, and control preprocessing. You also have the ability to inspect training statistics and results and perform parallel computations.

# **Table Content**


<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li><a href="https:">Introduction</a></li>
        <li><a href="#downloading_data">Data Processing</a></li>
        <li><a href="#evaluation">Auto Model Training and Evaluation
        <ul>
        <li><a href="#evaluation">Pycaret</a></li>
        <li><a href="#evaluation">Auto ML GS</a></li>
        <li><a href="#evaluation">MLflow</a></li>
         <li><a href="#evaluation">AutoML</a></li>
         <li><a href="#evaluation">KNN</a></li>
         <li><a href="#evaluation">Naive Bayes</a></li>
         <li><a href="#evaluation">Deep Neural Multilayer Perceptron (MLP)</a></li>
         <li><a href="#evaluation">Gradient Boosting</a></li>
         <li><a href="#evaluation">ExtraTreesClassifier</a></li>
         <li><a href="#evaluation">LGBM Classifer</a></li>
         <li><a href="#evaluation">CatBoost</a></li>
    </ul>
</div>
<br>
<hr>


# **1- Data Propcessing** 

Data preprocessing is an integral step in Machine Learning as the quality of data and the useful information that can be derived from it directly affects the ability of our model to learn; therefore, it is extremely important that we preprocess our data before feeding it into our model.

## **Import libaray**

In [12]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from sklearn.metrics import roc_curve, auc
from mlxtend.plotting import plot_confusion_matrix 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings("ignore")
from pycaret.utils import enable_colab
enable_colab()

Colab mode enabled.


## **Data Loading**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd 
Train_data = pd.read_csv("/content/drive/MyDrive/Datasets/Student field Recommendation /Placement_Data_Full_Class.csv")

In [6]:
Train_data

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0
1,2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0
2,3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0
3,4,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,NaN
4,5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,211,M,80.60,Others,82.00,Others,Commerce,77.60,Comm&Mgmt,No,91.0,Mkt&Fin,74.49,Placed,400000.0
211,212,M,58.00,Others,60.00,Others,Science,72.00,Sci&Tech,No,74.0,Mkt&Fin,53.62,Placed,275000.0
212,213,M,67.00,Others,67.00,Others,Commerce,73.00,Comm&Mgmt,Yes,59.0,Mkt&Fin,69.72,Placed,295000.0
213,214,F,74.00,Others,66.00,Others,Commerce,58.00,Comm&Mgmt,No,70.0,Mkt&HR,60.23,Placed,204000.0


# **PyCaret**

PyCaret is an open-source, low-code machine learning library and end-to-end model management tool built-in Python for automating machine learning workflows. It is incredibly popular for its ease of use, simplicity, and ability to build and deploy end-to-end ML prototypes quickly and efficiently.PyCaret is an alternate low-code library that can be used to replace hundreds of lines of code with few lines only. This makes the experiment cycle exponentially fast and efficient [1]

PyCaret is simple and easy to use. All the operations performed in PyCaret are sequentially stored in a Pipeline that is fully automated for deployment. Whether it’s imputing missing values, one-hot-encoding, transforming categorical data, feature engineering, or even hyperparameter tuning, PyCaret automates all of it.

PyCaret is an open-source, low-code machine learning library and end-to-end model management tool built-in Python for automating machine learning workflows. It is known for its ease of use, simplicity, and ability to quickly and efficiently build and deploy end-to-end ML prototypes.
PyCaret is an alternate low-code library that can replace hundreds of code lines with few lines only. This makes the experiment cycle exponentially fast and efficient[4]

![](https://drive.google.com/uc?export=view&id=1qj-WAAtNj7hDl6-o0Jg3gGS4jR1IaCGz)

## **Installing Pycaret**

In [ ]:
#capture #suppresses the displays
# install the full version
!pip install pycaret[full]

In [21]:
!pip install pyyaml==5.4.1


By installing the full version of pycaret, all the optional dependencies as listed here are also installed.

##**Import the necessary packages**

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pycaret
#from pycaret.regression import*
from pycaret.classification import*


## **Data Preparation** [3]

Common to all modules in PyCaret, the setup is the first and the only mandatory step in any machine learning experiment using PyCaret. This function takes care of all the data preparation required prior to training models. Besides performing some basic default processing tasks, PyCaret also offers a wide array of pre-processing features. To learn more about all the preprocessing functionalities in PyCaret, you can see this link.

In [9]:
Train_data

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.00,Mkt&HR,58.80,Placed,270000.0
1,2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.50,Mkt&Fin,66.28,Placed,200000.0
2,3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.00,Mkt&Fin,57.80,Placed,250000.0
3,4,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.00,Mkt&HR,59.43,Not Placed,NaN
4,5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.80,Mkt&Fin,55.50,Placed,425000.0
5,6,M,55.00,Others,49.80,Others,Science,67.25,Sci&Tech,Yes,55.00,Mkt&Fin,51.58,Not Placed,NaN
6,7,F,46.00,Others,49.20,Others,Commerce,79.00,Comm&Mgmt,No,74.28,Mkt&Fin,53.29,Not Placed,NaN
7,8,M,82.00,Central,64.00,Central,Science,66.00,Sci&Tech,Yes,67.00,Mkt&Fin,62.14,Placed,252000.0
8,9,M,73.00,Central,79.00,Central,Commerce,72.00,Comm&Mgmt,No,91.34,Mkt&Fin,61.29,Placed,231000.0
9,10,M,58.00,Central,70.00,Central,Commerce,61.00,Comm&Mgmt,No,54.00,Mkt&Fin,52.21,Not Placed,NaN


In [ ]:
from pycaret.regression import *
s = setup(Train_data, target = 'specialisation', transform_target = True, log_experiment = True, experiment_name = 'diamond')

When you initialize the setup function in PyCaret, it profiles the dataset and infers the data types for all input features. If all data types are correctly inferred, you can press enter to continue.[4]

Notice that:[4]

- I have passed log_experiment = True and experiment_name = 'diamond' , this will tell PyCaret to automatically log all the metrics, hyperparameters, and model artifacts behind the scene as you progress through the modeling phase. This is possible due to integration with MLflow.

- Also, I have used transform_target = True inside the setup. PyCaret will transform the Price variable behind the scene using box-cox transformation. It affects the distribution of data in a similar way as log transformation (technically different). If you would like to learn more about box-cox transformations, you can refer to this link.

## **Datasets**

In [ ]:
from pycaret.datasets import get_data
all_datasets = get_data()

In [ ]:
df = get_data('anomaly')
df.head()

## **Classificaiton**

**Model Training & Selection**

Now that data is ready for modeling, let’s start the training process by using compare_models function. It will train all the algorithms available in the model library and evaluates multiple performance metrics using k-fold cross-validation.*italicized text*

In [10]:
from pycaret.classification import *

We use ‘pycaret.classification’ to set up with original dataset, target, and session_id. After we do it, we compare any models.

In [ ]:
clf = setup(Train_data, target = 'specialisation', session_id=123, log_experiment = True, experiment_name = 'diamond') 

setup function in PyCaret, profiles the dataset and infers the data types for all input features. It is the first and the only mandatory step to start any machine learning experiment in PyCaret.

### **Model Evulaiton**

This function trains all the available models in the model library using default hyperparameters and evaluates performance metrics using cross-validation. The number of folds can be defined using the foldparameter (default = 10 folds). The table is sorted (highest to lowest) by the metric of choice which can be defined using the sortparameter(in this case we have sorted it on RMSE)
n_select parameter in the setup function controls the return of trained models. In this case, I am setting it to 15, meaning return the top 15 models as a list. pull function in the second line stores the output of compare_models as pd.DataFrame .

In [ ]:
compare_models()

### **Select Best Model**

In [ ]:
best= compare_models(n_select = 2, sort= 'Accuracy')

In [ ]:
compare_model_result = pull()



since the RandomForestregressor was evaluated to have a comparatively better Accuracy, let us build the model using RandomForestregressor

**Build the model using Best MODEL**

In [ ]:
lda = create_model('lda')

### **Dashboard**

In [ ]:
dashboard(lda, display_format='inline')

### **Hyperparameter tuning**

we need to improve it with tune_model. It is a function that automatically tunes the model with hyperparameters.

In [ ]:
tuned_lda= tune_model(lda, optimize='Accuracy', search_library='optuna')

Optimum Hyperparameters selected by using Optuna


In [ ]:
tuned_lda

### **Plot a Model Results**

After we make it, let’s create charts for analyzing the model.

**Analyze the Model Performance**


In [ ]:
plot_model(tuned_lda)

**Interpret the results**


In PyCaret, we can interpret the model by SHAP values and correlation plot with just one line of code

In [ ]:
interpret_model(tuned_lda)

**Evaluate the Model**


If we do not want to plot all these visualizations individually, then the PyCaret library has another amazing function — evaluate_model. In this function, you just need to pass the model object and PyCaret will create an interactive window for you to see and analyze the model in all the possible ways:

In [ ]:
evaluate_model(tuned_lda)

**AUC Plot**

In [ ]:
plot_model(tuned_lda, plot = 'auc')

**Precision-Recall Curve**

In [ ]:
plot_model(rf, plot = 'pr')

**check the residuals of trained model**


In [ ]:
plot_model(best, plot = 'residuals_interactive')

**Confusion Matrix**

In [ ]:
plot_model(rf, plot = 'confusion_matrix')

**Feature Importance Plot**

In [ ]:
plot_model(rf, plot='feature')

**Cross-validation**

Evaluate the model on the holdout set used for validation


In [ ]:
val_rf_pred = predict_model(tuned_rf)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,0.9789,0.9978,1.0,0.9599,0.9795,0.9578,0.9586


The predictions (Label) made is as follows:-


In [ ]:
val_rf_pred

In [ ]:
y_test

,RESULT_TEXT
9662,1.0
14694,1.0
2120,0.0
11684,1.0
724,1.0
...,...
20342,1.0
21213,1.0
5755,1.0
9849,1.0


### **Make predictions on test data**

We are almost done! We finalize the model and predict it with the original dataset. Let’s read the final dataset. As we can see, it has added labels and scores.

**Make predictions on test data**


In [ ]:
# predictions on new datasetRESULT_TEXT
pred_new_rf = predict_model(tuned_rf, data = X_test) #new_data is pd dataframe
#pred_new_rf = pred_new_rf.rename(columns={‘RESULT_TEXT’:’prediction’})
pred_new_rf = pred_new_rf.rename(columns={'RESULT_TEXT':'Prediction'})
pred_new_rf

In [ ]:
rf_final = finalize_model(rf)
predict_rf = predict_model(rf_final,data)

predict_rf.head()

RandomForestRegressor submission Scores(RMSE) : 4234.403588306586.


### **Finalize and Save Pipeline**

**Finalize and Save Pipeline**

Let’s now finalize the best model i.e. train the best model on the entire dataset including the test set and then save the pipeline as a pickle file.

Caution: One final word of caution. Once the model is finalized using finalize_model(), the entire dataset including the test/hold-out set is used for training. As such, if the model is used for predictions on the hold-out set after finalize_model() is used, the information grid printed will be misleading as you are trying to predict on the same data that was used for modeling. In order to demonstrate this point only, we will use final_rf under predict_model() to compare the information grid with the one above in section 11


In [ ]:
# finalize the model
final_best = finalize_model(rf)

In [ ]:
final_rf = finalize_model(tuned_rf)

In [ ]:
# save model to disk
save_model(final_best, 'diamond-pipeline')

save_model function will save the entire pipeline (including the model) as a pickle file on your local disk. By default, it will save the file in the same folder as your Notebook or script is in but you can pass the complete path as well if you would like:

In [ ]:
save_model(final_best, 'E:/Major Revision/finarf'

In [ ]:
# import classification module and setup environmentfrom pycaret.classification import *
clf1 = setup(dataset, target = 'left', silent = True)# train and save xgboost modelxgboost = create_model('xgboost', verbose = False)
final_xgboost = finalize_model(xgboost)
save_model(final_xgboost, 'C:/Users/username/xgboost_powerbi')

## **Regression**

**Model Training & Selection**

In [ ]:
from pycaret.regression import *
s = setup(MiceImputed, target = 'RESULT_TEXT', transform_target = True, log_experiment = True, experiment_name = 'diamond')

In [ ]:
compare_models()


""


[]

**Select Best Model**



In [ ]:
best= compare_models(n_select = 2, sort= 'RMSE')


""


### **Building Ensemble Models using PyCaret**

Ensemble models in machine learning combine the decisions from multiple models to improve the overall performance. In PyCaret, we can create bagging, boosting, blending, and stacking ensemble models with just one line of code. Here we will try with blending.
Blending models is a method of ensembling which uses consensus among estimators to generate final predictions. The idea behind blending is to combine different machine learning algorithms and use a majority vote or the average predicted probabilities in case of classification to predict the final outcome.

In [ ]:
bagged_dt = ensemble_model(dt, method = 'Bagging')

In [ ]:
# train a voting regressor dynamically
blender_specific = blend_models(estimator_list = compare_models(n_select = 5))

The calibrated blending model is as follows:-


In [ ]:
blender_specific

**Analyzing model performance in PyCaret**

In [ ]:
plot_model(blender_specific)

**Evaluate the model performance**


In [ ]:
evaluate_model(blender_specific)

**Make predictions on the validation set**


In [ ]:
val_pred = predict_model(blender_specific)

**Make Predictions on unseen data**


In [ ]:
# predictions on new dataset
pred_new = predict_model(blender_specific, data = X_train) #new_data is pd dataframe
pred_new = pred_new.rename(columns={‘RESULT_VALUE’:'prediction'})
pred_new

**Save and Load the Model**


In [ ]:
save_model(blender_specific , ‘/content/drive/MyDrive/blender_specific_saved_06142021’)
# Loading the saved model
blend_saved = load_model(‘/content/drive/MyDrive/blender_specific_saved_06142021’)
pred = blend_saved.predict(test)
pred[:5]

## **Clustering** 

###  **Method 1**

In [ ]:
cluster = setup(Train_data, session_id = 7652)

In [ ]:
model = create_model('kmeans')

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.6702,569.4594,0.4276,0,0,0


In [ ]:
plot_model(model, 'elbow')

In [ ]:
model = create_model('kmeans', num_clusters = 3)

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.7227,405.3411,0.482,0,0,0


In [ ]:
plot_model(model, 'cluster')

In [ ]:
save_model(model, 'clustering_model')
results = assign_model(model)
results.head(10)

### **Method 2**

In [ ]:
from pycaret.clustering import *

In [ ]:
dataset = get_clusters(Train_data, model = 'kmodes', num_clusters = 3)

In [ ]:
dataset

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary,Cluster
0,1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0,Cluster 0
1,2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0,Cluster 2
2,3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0,Cluster 2
3,4,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,NaN,Cluster 1
4,5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0,Cluster 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,211,M,80.60,Others,82.00,Others,Commerce,77.60,Comm&Mgmt,No,91.0,Mkt&Fin,74.49,Placed,400000.0,Cluster 0
211,212,M,58.00,Others,60.00,Others,Science,72.00,Sci&Tech,No,74.0,Mkt&Fin,53.62,Placed,275000.0,Cluster 0
212,213,M,67.00,Others,67.00,Others,Commerce,73.00,Comm&Mgmt,Yes,59.0,Mkt&Fin,69.72,Placed,295000.0,Cluster 0
213,214,F,74.00,Others,66.00,Others,Commerce,58.00,Comm&Mgmt,No,70.0,Mkt&HR,60.23,Placed,204000.0,Cluster 0


## **Anomaly Detection**

Anomaly Detection is a machine learning technique used for identifying rare items, events, or observations by checking for rows in the table that differ significantly from the majority of the rows. Typically, the anomalous items will translate to some kind of problem such as bank fraud, a structural defect, medical problem or error. Some common business use cases for anomaly detection are:
✔ Fraud detection (credit cards, insurance, etc.) using financial data.
✔ Intrusion detection (system security, malware) or monitoring for network traffic surges and drops.
✔ Identifying multivariate outliers in the dataset.
[13]

In [1]:
import pandas as pd 
data = pd.read_csv("/content/drive/MyDrive/Datasets/Anomaly Detection/anomaly.csv")

### **Model Creation**

Choose the optimal model from the model library and create models for anomaly detection. We can display the list of models by using the model() function.[14]

In [ ]:
from pycaret.anomaly import *
setup = setup(df, session_id = 123)

In [ ]:
models()

Specify the session id, this results in processing after execution. It interprets numerous types of variables automatically and allows us to confirm by pressing ENTER to continue.

Observe that our dataset consists of 10 features, 1000 rows each. We can perform various imputations- numeric and categorical or normalize the data. But we don’t require such transformations in our dataset so let us continue!

Performing all these computations with a few lines of code presents the beauty of the PyCaret library[14]

In [2]:
! pip install scikit-learn==0.23.2

In [ ]:
from pycaret.anomaly import *
dataset = get_outliers(data = data)

In [ ]:
dataset

Two new columns are attached to the original table. Label (1 = outlier, 0 = inlier) and Score (data points with high scores are categorized as outlier).

By default, PyCaret trains a K-Nearest Neighbors Anomaly Detector with 5% fraction (i.e. 5% of the total number of rows in the table will be flagged as outlier). Default values can be changed easily:

To change the fraction value you can use fraction parameter within get_outliers( ) function.
To change model type use model parameter within get_outliers( ).
See the following code for training an Isolation Forest model with 0.1 fraction:[13]

In [6]:
from pycaret.anomaly import *
dataset = get_outliers(dataset, model = 'iforest', fraction = 0.1)

In [ ]:
dataset

There are over 10 ready-to-use anomaly detection algorithms in PyCaret:

![](https://drive.google.com/uc?export=view&id=11lZ2xTuK4X7u5NlMxQ23hZgTo53pDZ1D)

All the preprocessing tasks necessary to train an anomaly detection model such as missing value imputation (if table has any missing or null values), or normalization, or one-hot-encoding, they all are automatically performed before training an anomaly detection model. Click here to learn more about PyCaret’s preprocessing capabilities.

💡 In this example we have used the **get_outliers( )** function to assign outlier label and score for analysis. Every time the query is refreshed, outliers are recalculated. An alternate way to implement this would be to use the predict_model( ) function to predict outliers using a pre-trained model in Python [13]

## **Natural Language Processing**

Several techniques are used to analyze text data among which Topic Modeling is a popular one. A topic model is a type of statistical model for discovering the abstract topics in a collection of documents. Topic modeling is a frequently used text-mining tool for the discovery of hidden semantic structures in a text data[13]

In [12]:
import pandas as pd 
dataset = pd.read_csv("/content/drive/MyDrive/Datasets/NLP/kiva.csv")

In [10]:
from pycaret.nlp import *
dataset = get_topics(data = dataset, text = 'en')

Description,Value
session_id,545
Documents,6818
Vocab Size,10671
Custom Stopwords,False


In [ ]:
dataset

‘en’ is the name of the column containing text in the table ‘kiva’.

Once the code is executed, new columns with weight of topics and dominant topic are attached to the original table. There are many ways to visualize the output of Topic Mod

By default, PyCaret trains a Latent Dirichlet Allocation model with 4 topics. Default values can be changed easily:

To change the number of topics you can use the num_topics parameter within get_topics( ) function.
To change model type use the model parameter within the get_topics( ).
See the example code for training a Non-Negative Matrix Factorization Model with 10 topics:[13]

In [13]:
from pycaret.nlp import *
dataset = get_topics(dataset, 'en', model = 'nmf', num_topics = 10)

Description,Value
session_id,2284
Documents,6818
Vocab Size,10671
Custom Stopwords,False


In [ ]:
dataset

PyCaret has following ready-to-use algorithms for topic modeling:



![](
https://drive.google.com/uc?export=view&id=1zKKxIIy1O8ZGDeg01ZuEitf8KRXNTag4)

## **Association Rule Mining**

Association Rule Mining is a rule-based machine learning technique for discovering interesting relations between variables in a database. It is intended to identify strong rules using measures of interestingness. Some common business use cases for association rule mining are:

✔ Market Basket Analysis to understand items frequently bought together.

✔ Medical Diagnosis to assist physicians in determining occurrence probability of illness given factors and symptoms.
[13]

In [15]:
import pandas as pd 
dataset = pd.read_csv("/content/drive/MyDrive/Datasets/Association Rule Mining/france.csv")

**Apriori Algorithm**

In [ ]:
from pycaret.arules import *
dataset = get_rules(dataset, transaction_id = 'InvoiceNo', item_id = 'Description')

In [17]:
dataset

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(JUMBO BAG WOODLAND ANIMALS),(POSTAGE),0.0651,0.6746,0.0651,1.0000,1.4823,0.0212,inf
1,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...",(SET/6 RED SPOTTY PAPER CUPS),0.0868,0.1171,0.0846,0.9750,8.3236,0.0744,35.3145
2,"(SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...",(SET/6 RED SPOTTY PAPER PLATES),0.0868,0.1085,0.0846,0.9750,8.9895,0.0752,35.6616
3,"(SET/6 RED SPOTTY PAPER CUPS, POSTAGE, SET/20 ...",(SET/6 RED SPOTTY PAPER PLATES),0.0716,0.1085,0.0694,0.9697,8.9406,0.0617,29.4208
4,"(SET/6 RED SPOTTY PAPER PLATES, POSTAGE, SET/2...",(SET/6 RED SPOTTY PAPER CUPS),0.0716,0.1171,0.0694,0.9697,8.2783,0.0610,29.1345
...,...,...,...,...,...,...,...,...,...
136,(STRAWBERRY LUNCH BOX WITH CUTLERY),(LUNCH BOX WITH CUTLERY RETROSPOT ),0.1041,0.1236,0.0542,0.5208,4.2124,0.0414,1.8289
137,(LUNCH BAG APPLE DESIGN),(LUNCH BAG SPACEBOY DESIGN ),0.1085,0.1041,0.0564,0.5200,4.9942,0.0451,1.8664
138,(LUNCH BAG APPLE DESIGN),(LUNCH BAG RED RETROSPOT),0.1085,0.1323,0.0564,0.5200,3.9298,0.0420,1.8077
139,"(LUNCH BAG RED RETROSPOT, POSTAGE)",(LUNCH BAG APPLE DESIGN),0.1041,0.1085,0.0521,0.5000,4.6100,0.0408,1.7831


# **AutoML-Gs**

**Installing required libraries**


In [ ]:
!pip install automl-gs


**Importing required libraries**


In [ ]:
import automl_gs
from automl_gs import automl_grid_search


**Creating the Model**


In [ ]:
automl_grid_search(MiceImputed, 'RESULT_TEXT')


# **MLflow**[8]

MLflow provides a convenient way to build end-to-end Machine Learning pipelines in production and in this guide

MLflow is an open-source platform for managing the end-to-end machine learning lifecycle or pipeline. It supports multiple Machine Learning libraries, algorithms, deployment tools, and programming languages.

The platform was created by Databricks has over 10,000 stars on GitHub with over 300+ contributors updating the platform on a daily basis.

The MLflow platform provides four major components:


- **MLflow Tracking** — This component is perfect for users looking to select the right hyper-parameters for their model as well as for recording model performance over time. You can record and query model experiments which include your code, data, config, and results.
- **MLflow Projects** — This component is mostly used when trying to reproduce your Machine Learning project on a different machine (similar to Docker images and containers). You can package your data science code in a given format to reproduce model runs on any platform.
- **MLflow Models** — This component is for you if you are looking to deploy Machine Learning models in the real world. With this component, you can deploy machine learning models in diverse serving environments.
- **MLflow Model Registry**— This component is great for you if you are looking to manage multiple Machine Learning models. Using this component, you can store, annotate, discover, and manage Machine Learning models in a central repository.

![](https://drive.google.com/uc?export=view&id=1BoZqaCplUEEJ8fDpk3k167WGi_GmJI_f)

- Individual Data Scientists can use MLflow to track experiments locally, organize code in projects for future reuse, and deploy models using MLflow’s deployment tools.
- Large Organizations can share projects, models, and results with any teams using MLflow.
- Data Science Teams can log metrics and compare results across multiple users working on the same problem
- Production Engineers can deploy models from diverse ML libraries, store the models as files in a management system, and track which run a model came from.

Based on these components, MLflow is designed to be useful for an individual to a large range of people working as a team. Some of its applications are as follows:

If you’re still undecided about learning how to use the platform, you can go over the MLflow components again and figure out if the platform is for you or not.

**Installing MLflow to use with Python**


**Using with Python [4]**

Remember we passed log_experiment = True in the setup function along with experiment_name = 'diamond' . Let’s see the magic PyCaret has done with the help of MLflow behind the scene. To see the magic let’s initiate the MLflow server:

In [ ]:
# within notebook (notice ! sign infront)
!mlflow ui
# on command line in the same folder
#mlflow ui

In [ ]:
!mlflow ui --host 0.0.0.0

[2021-11-01 12:02:00 +0000] [5949] [INFO] Starting gunicorn 20.1.0
[2021-11-01 12:02:00 +0000] [5949] [INFO] Listening at: http://0.0.0.0:5000 (5949)
[2021-11-01 12:02:00 +0000] [5949] [INFO] Using worker: sync
[2021-11-01 12:02:00 +0000] [5952] [INFO] Booting worker with pid: 5952
[2021-11-01 12:06:48 +0000] [5949] [INFO] Handling signal: int
[2021-11-01 12:06:48 +0000] [5952] [INFO] Worker exiting (pid: 5952)

Aborted!
[2021-11-01 12:06:49 +0000] [5949] [INFO] Shutting down: Master


Now open your browser and type “localhost:5000”. It will open a UI like this:

Remember that MLflow supports multiple programming languages and tools such as R-programming language or Python. It also comes with a graphical user interface that you can access from your browser once you successfully install MLflow.[8]

To keep this guide concise and easy to digest, we’ll show you how you can install MLflow to use with Python. To install MLflow, open up your command line/terminal and write the following command:

In [ ]:
!pip install mlflow

Note: You must have Python installed in your system to use pip which is Python’s package manager.

Once you execute the command, MLflow will get installed in your system. You can check if the installation is successful or not by importing MLflow in Python using the following line of code:

In [ ]:
import mlflow

If this line of Python code doesn’t give you an error, then, you’ve successfully installed MLflow to use with Python.

**Component 1: MLflow Tracking**


In [ ]:
# Importing the os library to work with operating system functionalities
import os

# Importing tracking functions from MLflow
from mlflow import log_metric, log_param, log_artifacts

# Logging a parameter (key-value pair)
log_param("param1", 0)

# Logging a metric; metrics can be updated throughout the run
log_metric("foo", 100)
log_metric("foo", 200)
log_metric("foo", 300)

# Create a file called test.text in outputs directory
if not os.path.exists("outputs"):
    os.makedirs("outputs")
with open("outputs/test.txt", "w") as f:
    f.write("hello world!")

# Logging an artifact (output file)
log_artifacts("outputs")

In [ ]:
!mlflow ui

# **AutoML**[9]

An AutoML pipeline includes data preprocessing, feature engineering, feature selection, model training, hyperparameter tuning, and algorithm selection

What if there was an open-source Python package that could do all of the above and automatically determine the best model? A package that would explain to you exactly how each ML model is built & also provide you with a detailed report!
The answer to the question is the Python package ‘mjlar supervised’. It was developed by MJLAR with the goal of helping software developers and data analysts that do not have classical ML training.

The package ‘mjlar supervised’allows you to select machine learning algorithms, preprocess dataset, train & tune the model, and finally explain and evaluate the model.

The idea behind AutoML is to continuously build and test machine learning models using a neural network. As a new set of data is added, the neural network will automatically determine whether it needs to create a new specialized model by training and executing its own algorithm or simply improve an existing one. Essentially, AutoML creates sophisticated machine learning programs that can be accessed via APIs where the output is returned in the form of API calls.
AutoML is clearly a very powerful tool for automating much of this process using machine learning algorithms. But you’re still going to need programmers to define heuristics for machines or identify how they should make decisions and also learn about various data science concepts.[10]

**Why AutoML?**

AutoML would continuously build models based on new data sets provided by an enterprise and improve them when better ones are discovered by the AutoML algorithm. The idea of applying neural networks to this process is not new — it’s been around for a long time and several companies have experimented with it in their own ways. AutoML has tremendous implications for non-technical people who want to leverage insights from data, but don’t have the programming chops or the resources to implement ML solutions. All they need now is an API call where they can get data back in the form of “recommendations”, such as which products are ideal for each customer based on his purchasing history, demographics, and other aspects. The other possibility is that they will get an alert if a certain event happens and the solution to prevent it.[10]

**What Does AutoML Do?**[10]


AutoML can be used for solving three primary problems: Model Building, Hyperparameter Tuning & Selection, and Model Optimization.
Model Building — This refers to setting up the model based on parameters in your data set (such as predicting the probability of customers churning) so that machine learning algorithms can compare them with labels (actual instances) and then feedback their “correctness” or explain how inaccurate they are. Once modeled, results can be analyzed using tools like TensorBoard. The process of refining these models is known as Hyperparameter tuning & selection — this refers to setting up the appropriate parameters in a model so that it learns data efficiently. When you’re using AutoML, this is done by providing an algorithm with lots of examples and letting it come up with its own metrics for measuring what makes sense. For example, if you want a model that recognizes images — you’ll need to provide it with hundreds of thousands of images so it can identify patterns between them. Model Optimization refers to improving your existing models after training them on large datasets provided by an enterprise; AutoML helps optimize your models based on computation speed and accuracy.
AutoML has two powerful components — Abalone and Ranker (which have been open-sourced).
Abalone: This component takes care of creating a simple model of your data set, by using AutoML’s cloud platform that uses Tensorflow. It also helps you to build more advanced models with the help of human experts in machine learning.
Ranker: This component recommends which models (created via Abalone) are the best for a particular enterprise based on their usage requirements and results.

**Algorithm Selection**[9]

This package allows the user to check their model with various algorithms: Baseline, Linear, Random Forest, Extra Trees, LightGBM, Xgboost, CatBoost, Neural Networks, Ensemble, and Nearest Neighbors. Once you run the code, the output will show you how much time each model took to train the dataset and the root mean squared error of the trained model. The package will select the algorithm with the lowest root mean squared error.

**Available Modes**[9]

The available modes allow the user to explain and optimize the ML model according to the audience. It has four modes — Explain, Perform, Compete and Optuna.

In [ ]:
automl = AutoML(mode=”Explain”)

The Explain mode allows the user to explain the model including making learning curves, feature plots, and SHAP plots.

In [ ]:
automl = AutoML(mode=”Perform”)


The Perform mode allows the user to train a model that will be used in real-life use cases.

In [ ]:
automl = AutoML(mode=”Compete”)

The Compete mode will allow the user to adapt a validation strategy depending on the size of the dataset and can be used for entering into machine learning competitions.

In [ ]:
automl = AutoML(mode=”Optuna”)


The Optuna mode should be used when the priority is the performance of the model instead of time.

In [ ]:
!pip install mljar-supervised


In [ ]:
data=Train_data


In [ ]:
data['gender'] = data['gender'].fillna(data['gender'].mode()[0])
data['ssc_b'] = data['ssc_b'].fillna(data['ssc_b'].mode()[0])
data['hsc_b'] = data['hsc_b'].fillna(data['hsc_b'].mode()[0])
data['hsc_s'] = data['hsc_s'].fillna(data['hsc_s'].mode()[0])
data['degree_t'] = data['degree_t'].fillna(data['degree_t'].mode()[0])
data['workex'] = data['workex'].fillna(data['workex'].mode()[0])
data['specialisation'] = data['specialisation'].fillna(data['specialisation'].mode()[0])
data['status'] = data['status'].fillna(data['status'].mode()[0])

In [ ]:
# Convert categorical features to continuous features with Label Encoding
from sklearn.preprocessing import LabelEncoder
lencoders = {}
for col in data.select_dtypes(include=['object']).columns:
    lencoders[col] = LabelEncoder()
    data[col] = lencoders[col].fit_transform(data[col])

In [ ]:
X=data.drop('specialisation',axis=1)


In [ ]:
y=data[['specialisation']]


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=100)

In [ ]:
!pip uninstall scikit-learn -y
!pip install scikit-learn==0.24

Found existing installation: scikit-learn 0.24.0
Uninstalling scikit-learn-0.24.0:
  Successfully uninstalled scikit-learn-0.24.0
  Using cached scikit_learn-0.24.0-cp37-cp37m-manylinux2010_x86_64.whl (22.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pycaret 2.3.5 requires scikit-learn==0.23.2, but you have scikit-learn 0.24.0 which is incompatible.
mljar-supervised 0.11.1 requires scikit-learn>=0.24.2, but you have scikit-learn 0.24.0 which is incompatible.
mljar-supervised 0.11.1 requires scipy>=1.6.1, but you have scipy 1.5.4 which is incompatible.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from supervised.automl import AutoML
from sklearn.metrics import accuracy_score


In [ ]:
automl = AutoML()
automl.fit(X_train, y_train)

AutoML directory: AutoML_1
The task is binary_classification with evaluation metric logloss
AutoML will use algorithms: ['Baseline', 'Linear', 'Decision Tree', 'Random Forest', 'Xgboost', 'Neural Network']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 3 models
1_Baseline logloss 0.681181 trained in 0.44 seconds
2_DecisionTree logloss 0.810283 trained in 11.08 seconds
3_Linear logloss 0.719179 trained in 4.13 seconds
* Step default_algorithms will try to check up to 3 models
4_Default_Xgboost logloss 0.663438 trained in 6.67 seconds
5_Default_NeuralNetwork logloss 0.816192 trained in 1.14 seconds
6_Default_RandomForest logloss 0.707791 trained in 9.04 seconds
* Step ensemble will try to check up to 1 model
Ensemble logloss 0.654952 trained in 0.57 seconds
AutoML fit time: 50.75 seconds
AutoML best model: Ensemble


AutoML()

In [ ]:
predictions = automl.predict_all(X_test)
print(predictions.head())
print("Test accuracy:", accuracy_score(y_test, predictions["label"].astype(int)))

   prediction_0  prediction_1  label
0      0.402709      0.597291      0
1      0.395411      0.604589      0
2      0.381083      0.618917      1
3      0.381150      0.618850      1
4      0.391650      0.608350      1
Test accuracy: 0.5846153846153846


In [ ]:
automl = AutoML(mode="Perform")
automl.fit(X_train, y_train)

In [ ]:
# compute the accuracy on test data
predictions = automl.predict_all(X_test)
print(predictions.head())
print("Test accuracy:", accuracy_score(y_test, predictions["label"].astype(int)))

   prediction_0  prediction_1  label
0      0.497490      0.502510      0
1      0.363188      0.636812      1
2      0.289308      0.710692      1
3      0.315148      0.684852      1
4      0.530294      0.469706      0
Test accuracy: 0.6153846153846154


In [ ]:
!pip3 install autokeras


In [ ]:
from autokeras import StructuredDataClassifier

In [ ]:
search = StructuredDataClassifier(max_trials = 20)


In [ ]:
search.fit(x=X_train, y=y_train)


In [ ]:
loss, acc = search.evaluate(X_train, y_train, verbose=0)
print('Accuracy is {}'.format(acc))

Accuracy is 0.6600000262260437


In [ ]:
loss, acc = search.evaluate(X_test, y_test, verbose=0)
print('Accuracy is {}'.format(acc))

Accuracy is 0.6615384817123413


# **References**

[1-Build a machine learning model with PyCaret and corresponding user interface with Gradio](https://medium.com/nerd-for-tech/build-a-machine-learning-model-with-pycaret-and-corresponding-user-interface-with-gradio-57ff09b7d262)

[2-PyCaret in Machine Learning](https://thecleverprogrammer.com/2021/03/07/pycaret-in-machine-learning/)

[3-PyCaret 101: An introduction for beginners](https://www.kdnuggets.com/2021/06/pycaret-101-introduction-beginners.html?fbclid=IwAR29t4C5QplspexoIyAd4eYfUjlZSOFL5wVmxc9sU5qXr_mMqIPfaJVmP6Y)

[4-Easy MLOps with PyCaret + MLflow](https://towardsdatascience.com/easy-mlops-with-pycaret-mlflow-7fbcbf1e38c6)

[5-Binary Classification Tutorial Level Beginner](https://colab.research.google.com/drive/1GqQ3XAIzg4krBbnOpKyeRqT0qBQhdwYL#scrollTo=GM-nQ7LqEQma)

[6-Create Clusters](https://pycaret.org/create-clusters/)

[7-Plot Model](https://pycaret.org/plot-model/)

[8-MLflow For Machine Learning Pipelines [Ultimate Guide]](https://medium.com/@theclickreader/mlflow-for-machine-learning-pipelines-ultimate-guide-821e55370034)

[9-Gentle Introduction to New AutoML Package by MJLAR!](https://medium.com/mlearning-ai/gentle-introduction-to-new-automl-package-by-mjlar-bc51a99ba53c)

[10-AutoML: The future of Data Science and Machine Learning](https://protonautoml.medium.com/automl-the-future-of-data-science-and-machine-learning-45abb8f5ebcf)

[11- Predicting Machine Maintenance using Machine Learning](https://medium.com/analytics-vidhya/predicting-machine-maintenance-using-machine-learning-ee694c7fa0f0)


[12- 7 Top AutoML Tools](https://victorzhou.com/posts/tools-for-auto-ml/)

[13-Machine Learning in Power BI using PyCaret](https://www.kdnuggets.com/2020/05/machine-learning-power-bi-pycaret.html)


[Introduction to Anomaly Detection in Python with PyCaret](https://towardsdatascience.com/introduction-to-anomaly-detection-in-python-with-pycaret-2fecd7144f87)

[How to Use pyGAM to Fit More Flexible Functions to Your Data](https://tech-at-kraftheinz.medium.com/how-to-use-pygam-to-fit-more-flexible-functions-to-your-data-5de16e7d913a)

[Automating Machine Learning Modelling](https://towardsdatascience.com/automating-machine-learning-modelling-62aac2081e3f)

[What Are Baseline Models and Benchmarking For Machine Learning, Why We Need Them? Part 1 Classification](https://pub.towardsai.net/what-are-baseline-models-and-benchmarking-for-machine-learning-why-we-need-them-affe0714cd07)

[INTRODUCTION TO MACHINE LEARNING PIPELINES](https://t-dab.medium.com/introduction-to-machine-learning-pipelines-5218f7964f50)

[Automating Machine Learning Modelling](https://towardsdatascience.com/automating-machine-learning-modelling-62aac2081e3f)
